In [2]:
# make gene sets using http://www.go2msig.org/cgi-bin/go2msig.cgi
# and then reformat them to use uniprot IDs

In [3]:
import os
import requests
import pandas as pd
from tqdm import tqdm
from collections import defaultdict, Counter
from glob import glob
import numpy as np
from itertools import chain
import pandas as pd
import matplotlib
matplotlib.rcParams.update({'font.size': 22})

In [5]:
# get go mapping, so we can get the symbol to id mappings
df = pd.read_csv("../my_annotations.gaf", sep="\t", comment="!", header=None)
df = df.rename(columns={1:"id", 2:"symbol", 4:"go_id", 8:'aspect', 12:"taxon"})
df = df.query("go_id != 'GO:0005575'")  # remove 'cellular component'
df[df.id == "P37476"].head(2)

,0,id,symbol,3,go_id,5,6,7,aspect,9,10,11,taxon,13,14,15,16
79988,UniProtKB,P37476,ftsH,NaN,GO:0005886,GO_REF:0000037,IEA,UniProtKB-KW:KW-1003,C,ATP-dependent zinc metalloprotease FtsH,FTSH_BACSU|ftsH|BSU00690,protein,taxon:224308,20170603,UniProt,NaN,NaN
79989,UniProtKB,P37476,ftsH,NaN,GO:0005886,GO_REF:0000039,IEA,UniProtKB-SubCell:SL-0039,C,ATP-dependent zinc metalloprotease FtsH,FTSH_BACSU|ftsH|BSU00690,protein,taxon:224308,20170603,UniProt,NaN,NaN


In [11]:
symbol_id_bs = dict()
symbol_id_pa = dict()
symbol_id_jurkat = dict()
for _,row in df.iterrows():
    if row.taxon == "taxon:9606":
        symbol_id_jurkat[row.symbol.upper()] = row.id
    elif row.taxon == 'taxon:224308':
        symbol_id_bs[row.symbol.upper()] = row.id
    elif row.taxon == "taxon:208964":
        symbol_id_pa[row.symbol.upper()] = row.id

In [12]:
### http://www.go2msig.org/cgi-bin/go2msig.cgi
# settings: Gene association data source: GO database
# taxon: 9606, 224308, or 208964
# evidence codes: all
# Propagate gene associations up ontology hierarchy: True
# genesets: CC
# Minimum geneset size: 5

In [ ]:
"""
/usr/local/bin/go2msig	-bioannodb 'dbi:mysql:bioannotation_042015'
-biouser gouser -biopass amigo -godb 'dbi:mysql:mygo_032015' -gouser gouser
-gopass amigo  -mingenes 5 -maxgenes 700 -format gmt -taxid 9606 -geneid
symbol -ontology cc -evidence all -termsource cache -cachefile
webversion_2015 -assocsource godb
"""

In [13]:
# 9606: http://www.go2msig.org/output/outfilea0e03fb7d3499ede654f8c92b97fed45.gmt
# 224308: http://www.go2msig.org/output/outfilebfc1204c3ea99b594c840eedceb430d6.gmt
# 208964: http://www.go2msig.org/output/outfile02b916a9faf65700342b9f28755c0acb.gmt

In [23]:
# convert IDs to unirpot and rewrite gmt file
with open("msigdb/224308_uniprot.gmt", "w") as f:
    for line in open("msigdb/224308.gmt"):
        header = line.strip().split("\t")[:2]
        ids = line.strip().split("\t")[2:]
        symbols = [symbol_id_bs[x] for x in ids if x in symbol_id_bs]
        print("\t".join(header + symbols), file=f)

In [24]:
with open("msigdb/208964_uniprot.gmt", "w") as f:
    for line in open("msigdb/208964.gmt"):
        header = line.strip().split("\t")[:2]
        ids = line.strip().split("\t")[2:]
        symbols = [symbol_id_pa[x] for x in ids if x in symbol_id_pa]
        print("\t".join(header + symbols), file=f)

In [25]:
with open("msigdb/9606_uniprot.gmt", "w") as f:
    for line in open("msigdb/9606.gmt"):
        header = line.strip().split("\t")[:2]
        ids = line.strip().split("\t")[2:]
        symbols = [symbol_id_jurkat[x] for x in ids if x in symbol_id_jurkat]
        print("\t".join(header + symbols), file=f)